In [1]:
import config
import requests
import datetime as dt
import json
from datetime import datetime as dt, date
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import os
import sys

symbol_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

In [4]:
df_daily = {}
for symbol in symbol_list:
    df_daily[symbol] = pd.read_csv("./stock_price/daily/" + symbol + ".csv")

for symbol in symbol_list:
    df_daily[symbol]['Date'] = pd.to_datetime(df_daily[symbol]['Date'], format='%Y-%m-%d')
    df_daily[symbol]['diff'] = df_daily[symbol]['adjusted_close'] - df_daily[symbol]['adjusted_close'].shift(-1)
    df_daily[symbol]['direction'] = df_daily[symbol]['diff'].apply(lambda x: 1 if x > 0 else 0)
    df_daily[symbol] = df_daily[symbol][['Date','adjusted_close','volume', 'direction']]
    df_daily[symbol]['Symbol'] = symbol

union_daily = pd.concat([df_daily[symbol] for symbol in symbol_list])
union_daily = union_daily.sort_values(by=['Symbol','Date'])

In [3]:
df_monthly = {}
for symbol in symbol_list:
    df_monthly[symbol] = pd.read_csv("./stock_price/monthly/" + symbol + ".csv")

for symbol in symbol_list:
    df_monthly[symbol]['Date'] = pd.to_datetime(df_monthly[symbol]['Date'], format='%Y-%m-%d')
    df_monthly[symbol]['diff'] = df_monthly[symbol]['adjusted_close'] - df_monthly[symbol]['adjusted_close'].shift(-1)
    df_monthly[symbol]['direction'] = df_monthly[symbol]['diff'].apply(lambda x: 1 if x > 0 else 0)
    df_monthly[symbol] = df_monthly[symbol][['Date','adjusted_close','volume', 'direction']]
    df_monthly[symbol]['Symbol'] = symbol

union_monthly = pd.concat([df_monthly[symbol] for symbol in symbol_list])
union_monthly = union_monthly.sort_values(by=['Symbol','Date'])

In [8]:
df_weekly = {}
for symbol in symbol_list:
    df_weekly[symbol] = pd.read_csv("./stock_price/weekly/" + symbol + ".csv")

for symbol in symbol_list:
    df_weekly[symbol]['Date'] = pd.to_datetime(df_weekly[symbol]['Date'], format='%Y-%m-%d')
    df_weekly[symbol]['diff'] = df_weekly[symbol]['adjusted_close'] - df_weekly[symbol]['adjusted_close'].shift(-1)
    df_weekly[symbol]['direction'] = df_weekly[symbol]['diff'].apply(lambda x: 1 if x > 0 else 0)
    df_weekly[symbol] = df_weekly[symbol][['Date','adjusted_close','volume', 'direction']]
    df_weekly[symbol]['Symbol'] = symbol

union_monthly = pd.concat([df_weekly[symbol] for symbol in symbol_list])
union_monthly = union_monthly.sort_values(by=['Symbol','Date'])

In [12]:
for symbol in symbol_list:
    stock_daily = df_daily[symbol]
    #stock_daily.rename(columns={'5. adjusted close':'adjusted_close','6. volume':'volume'},inplace=True)
    stock_daily.to_csv("encode_price/Daily/"+symbol+'.csv',index=0)
    stock_weekly = df_weekly[symbol]
    #stock_weekly.rename(columns={'5. adjusted close':'adjusted_close','6. volume':'volume'},inplace=True)
    stock_weekly.to_csv("encode_price/Weekly/"+symbol+'.csv',index=0)
    stock_monthly = df_monthly[symbol]
    #stock_monthly.rename(columns={'5. adjusted close':'adjusted_close','6. volume':'volume'},inplace=True)
    stock_monthly.to_csv("encode_price/Monthly/"+symbol+'.csv',index=0)

In [13]:
def calculate_sma(df, symbols, window=20):
    sma_df = df  # Initialize an empty DataFrame to store the results
    
    sma_df["SMA"] = df['adjusted_close'].rolling(window=window).mean()
    
    return sma_df

In [14]:
def calculate_ema(df, symbols, window=20):   
    sma_df = df  # Initialize an empty DataFrame to store the results
    
    sma_df["EMA"] = df['adjusted_close'].ewm(span=window, adjust=False).mean()
    
    return sma_df

In [15]:
###
def calculate_stoch(df, k_window=14, d_window=3): 
    
    # Filter the DataFrame to include only the rows with the specified symbol
    symbol_df = df
    
    # Set the date column as the DataFrame index
    symbol_df = symbol_df.set_index('Date')
    symbol_df['4. close'] = pd.to_numeric(symbol_df['4. close'])

    # Calculate the lowest and highest closing prices over the past k_window days
    low_min = symbol_df['3. low'].rolling(window=k_window).min()
    high_max = symbol_df['2. high'].rolling(window=k_window).max()
    
    # Calculate the %K line using the current closing price and the recent trading range
    k = 100 * (symbol_df['4. close']) - low_min / (high_max - low_min)
    
    # Calculate the %D line using a moving average of the %K line
    d = k.rolling(window=d_window).mean()
    
    # Combine the %K and %D lines into a DataFrame
    stoch_df = pd.DataFrame({'%K': k, '%D': d})
    
    return stoch_df

In [16]:
def calculate_rsi(df1, symbol, period=14):
    """
    Calculates the relative strength index (RSI) of a DataFrame.

    Parameters:
    period (int): Number of periods to use for the RSI calculation.

    Returns:
    pandas.DataFrame: DataFrame containing the RSI values.
    """
    df = df1
    df["4. close"] = pd.to_numeric(df["4. close"])
    delta = df['4. close'].diff()

    # Get upward and downward price movements
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the exponential moving average (EMA) of gains and losses
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    # Calculate the relative strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    # Add RSI values to the DataFrame
    df[symbol + '_rsi'] = rsi

    return df.loc[:,["Date", symbol + '_rsi']]

In [17]:
def calculate_mfi(df1, symbol, period=14):
    """
    Calculates the money flow index (MFI) of a DataFrame.

    Parameters:
    period (int): Number of periods to use for the MFI calculation.

    Returns:
    pandas.DataFrame: DataFrame containing the MFI values.
    """
    df = df1
    df["4. close"] = pd.to_numeric(df["4. close"])
    df["3. low"] = pd.to_numeric(df["3. low"])
    df["2. high"] = pd.to_numeric(df["2. high"])
    df["6. volume"] = pd.to_numeric(df["6. volume"])
    # Calculate typical price
    typical_price = (df['4. close'] + df['3. low'] + df['2. high']) / 3

    # Calculate raw money flow
    money_flow = typical_price * df['6. volume']

    # Get positive and negative money flow
    positive_flow = np.where(typical_price > typical_price.shift(1), money_flow, 0)
    negative_flow = np.where(typical_price < typical_price.shift(1), money_flow, 0)

    # Calculate the money flow ratio
    raw_mfr = positive_flow / negative_flow
    mfr = pd.Series(np.where(negative_flow == 0, 100, 100 - (100 / (1 + raw_mfr))))

    # Calculate the MFI
    mfi = mfr.rolling(window=period).mean()

    # Add MFI values to the DataFrame
    df[symbol + '_mfi'] = mfi

    return df.loc[:,["Date", symbol + '_mfi']]

In [18]:
def calculate_sar(df1, symbol, lookback=14, acceleration_factor=0.02):
    # Load the price data into a pandas DataFrame
    df = df1
    print(len(df))
    df["close"] = pd.to_numeric(df["4. close"])
    df["low"] = pd.to_numeric(df["3. low"])
    df["high"] = pd.to_numeric(df["2. high"])
    print(len(df))
    # Initialize the SAR for the first day as the lowest low over the lookback period
    sar = df['low'].rolling(window=lookback).min()
    trend = pd.Series(np.zeros(len(df)))

    # Loop through the data and calculate the SAR and trend for each day
    for i in range(1, len(df)):
        if df['low'][i] > sar[i-1]:
            trend[i] = 1
        elif df['high'][i] < sar[i-1]:
            trend[i] = -1
        else:
            trend[i] = trend[i-1]

        if trend[i] == 1:
            if sar[i-1] == df['low'][i-1]:
                sar[i] = sar[i-1] + acceleration_factor * (df['high'][i-1] - sar[i-1])
            else:
                sar[i] = sar[i-1] + acceleration_factor * (df['high'][i-1] - sar[i-1]) * trend[i-1]
            if sar[i] > df['low'][i]:
                sar[i] = df['low'][i]
        else:
            if sar[i-1] == df['high'][i-1]:
                sar[i] = sar[i-1] - acceleration_factor * (sar[i-1] - df['low'][i-1])
            else:
                sar[i] = sar[i-1] + acceleration_factor * (df['low'][i-1] - sar[i-1]) * trend[i-1]
            if sar[i] < df['high'][i]:
                sar[i] = df['high'][i]

    # Create a new DataFrame with the calculated Parabolic SAR values and the original price data
    sar_df = pd.DataFrame({'Parabolic SAR': sar}, index=df['Date'])
    
    return sar_df

In [19]:
def calc_AD(df):
    # Initialize the AD line as the closing price
    df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["4. close"])
    df["low"] = pd.to_numeric(df["3. low"])
    df["high"] = pd.to_numeric(df["2. high"])
    df["volume"] = pd.to_numeric(df["6. volume"])
    ad = df['close'].copy()
    
    # Calculate the money flow multiplier (MFM)
    mfm = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'])
    
    # Calculate the money flow volume (MFV)
    mf_volume = mfm * df['volume']
    
    # Calculate the cumulative sum of the money flow volume
    cmf = mf_volume.cumsum()
    
    # Calculate the AD line
    ad.iloc[0] = cmf.iloc[0]
    for i in range(1, len(ad)):
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            ad.iloc[i] = ad.iloc[i-1] + cmf.iloc[i]
        elif df['close'].iloc[i] < df['close'].iloc[i-1]:
            ad.iloc[i] = ad.iloc[i-1] - cmf.iloc[i]
        else:
            ad.iloc[i] = ad.iloc[i-1]
            
    # Set the index of the DataFrame to the date column
    ad.index = df['Date']
    
    return pd.DataFrame(ad)

In [20]:
def calculate_vwap(symbol, period):
    df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["4. close"])
    df["low"] = pd.to_numeric(df["3. low"])
    df["high"] = pd.to_numeric(df["2. high"])
    df["volume"] = pd.to_numeric(df["6. volume"])

    # Calculate the typical price for each period
    tp = (df['high'] + df['low'] + df['close']) / 3

    # Multiply the typical price by the volume for each period
    vtp = tp * df['volume']

    # Calculate the cumulative sum of the volume times the typical price
    cum_vtp = vtp.cumsum()

    # Calculate the cumulative sum of the volume
    cum_vol = df['volume'].cumsum()

    # Calculate the VWAP for each period
    vwap = cum_vtp / cum_vol

    # Create a DataFrame to hold the results
    results = pd.DataFrame({
        'vwap': vwap[period-1:],
        'date': df['Date'][period-1:]
    })

    # Set the index to the start time
    results.set_index('date', inplace=True)

    return results

In [21]:
def calculate_macd(symbol, ema_short=12, ema_long=26, signal_period=9):
    df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["4. close"])
    # Calculate the short-term and long-term EMAs
    ema_short = df['close'].ewm(span=ema_short, adjust=False).mean()
    ema_long = df['close'].ewm(span=ema_long, adjust=False).mean()
    
    # Calculate the MACD line and the signal line
    macd_line = ema_short - ema_long
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    
    # Calculate the MACD histogram
    macd_histogram = macd_line - signal_line
    
    # Return a DataFrame with the MACD values
    macd_df = pd.DataFrame({
        'MACD': macd_line,
        'Signal': signal_line,
        'Histogram': macd_histogram
    }, index=df.index)
    
    return macd_df
